## e009 EDA
 * group 内の preds を使う

In [1]:
import pandas as pd
import numpy as np
import gc
import pickle
import pyarrow.parquet as pq
from multiprocessing import Pool
from tqdm import tqdm_notebook as tqdm

import scipy
from scipy import signal
from scipy.signal import butter

from matplotlib import pyplot as plt
import seaborn as sns

import sys
sys.path.append('../../guchio_utils/')

In [2]:
#plt.rcParams['font.family'] = 'IPAPGothic' #全体のフォントを設定
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
# %config InlineBackend.figure_formats = {'png', 'retina'}
%load_ext autoreload
%autoreload 1

### OOF analysis

In [2]:
#trn_df = pd.read_parquet('../inputs/origin/train.parquet.zip', compression='zip')
trn_df = pq.read_pandas('../inputs/origin/train.parquet').to_pandas()
trn_meta_df = pd.read_csv('../inputs/origin/metadata_train.csv')

In [3]:
trn_hp_dn_df = pd.read_pickle('../inputs/prep/train_hp_dn.pkl.gz', compression='gzip')
trn_hp_dn_df.head()

,0,1,2,3,4,5,6,7,8,9,...,8702,8703,8704,8705,8706,8707,8708,8709,8710,8711
0,13.601562,0.000906,-14.601562,-14.359375,-4.300781,17.921875,-8.484375,6.421875,-0.000802,-13.218750,...,15.773438,-17.171875,10.335938,6.441406,11.609375,3.173828,-20.593750,-13.429688,-6.679688,17.515625
1,16.234375,0.000906,-17.515625,-17.390625,-5.820312,21.062500,-9.898438,9.070312,-0.000802,-15.539062,...,18.187500,-21.968750,12.976562,7.500000,14.734375,4.218750,-18.437500,-16.000000,-7.945312,22.015625
2,11.335938,0.000906,-12.445312,-14.546875,-4.878906,18.562500,-11.429688,12.156250,-0.000802,-12.531250,...,14.906250,-17.859375,11.609375,5.574219,13.398438,5.417969,-17.937500,-13.179688,-5.648438,20.375000
3,12.453125,0.000906,-13.718750,-14.234375,-4.585938,18.843750,-12.609375,14.265625,-0.000802,-11.726562,...,14.937500,-18.078125,11.570312,6.031250,13.648438,6.257812,-16.890625,-13.085938,-6.136719,21.187500
4,17.343750,0.000906,-18.937500,-13.914062,-3.957031,19.437500,-13.406250,15.304688,-0.000802,-11.187500,...,15.960938,-19.500000,11.343750,7.976562,13.796875,6.699219,-17.671875,-13.539062,-8.343750,21.234375


### 再度 oof 

In [10]:
# with open('../oofs/e007_2019-01-16-14-34-03_0.5397_0.17_oofs.pkl', 'rb') as fin:
with open('../oofs/e008_2019-01-16-17-15-02_0.5383_0.207_oofs.pkl', 'rb') as fin:
    oof = pickle.load(fin)

In [11]:
oof

[[array([   3,    4,    5, ..., 8709, 8710, 8711]),
  array([  15,   16,   17, ..., 8691, 8692, 8693]),
  array([  12,   13,   14, ..., 8706, 8707, 8708]),
  array([   0,    1,    2, ..., 8700, 8701, 8702]),
  array([   6,    7,    8, ..., 8697, 8698, 8699])],
 [array([0.14339742, 0.38784939, 0.36013803, ..., 0.2877849 , 0.0070246 ,
         0.02295375]),
  array([1.44399327e-04, 1.11547420e-05, 1.44386204e-04, ...,
         2.93916138e-05, 6.45865418e-06, 1.28585802e-05]),
  array([3.24142288e-04, 2.32358380e-02, 1.85473408e-03, ...,
         2.66883710e-05, 4.97293663e-05, 8.87931803e-06]),
  array([8.08961869e-02, 1.54722635e-02, 5.84360722e-03, ...,
         7.18891119e-06, 9.63243261e-06, 2.41244634e-05]),
  array([5.42586989e-04, 1.06259744e-03, 7.84617530e-05, ...,
         1.21294673e-05, 4.22545563e-05, 3.59573441e-05])]]

In [12]:
len(oof), len(oof[0]), len(oof[1]), 

(2, 5, 5)

In [75]:
def place_0(x):
    return x.iloc[0]

def place_1(x):
    return x.iloc[1]

def place_2(x):
    return x.iloc[2]

def y_preds_features(oofs, val_idxes):
    # Make the base df
    y_preds_df = pd.DataFrame()
    y_preds_df['signal_id'] = np.concatenate(val_idxes).astype(int)
    y_preds_df['y_pred'] = np.concatenate(oofs).astype('float64')
    y_preds_df['id_measurement'] = (y_preds_df['signal_id']//3).astype(int)
    y_preds_df.sort_values('signal_id', inplace=True)
    y_preds_df.reset_index(drop=True, inplace=True)
    
    # Agg
    agg_y_preds_df = y_preds_df.groupby('id_measurement').agg({
        'y_pred': ['max', 'min', 'mean', 'std', place_0, place_1, place_2]
    })
    agg_y_preds_df.columns = [e[0] + '_' + e[1] for e in agg_y_preds_df.columns]
    
   # Merge 
    y_preds_df = y_preds_df.merge(agg_y_preds_df, on='id_measurement', how='left').drop(['signal_id', 'id_measurement'], axis=1)
    return y_preds_df

In [76]:
y_preds_features(oof[1], oof[0])

,y_pred,y_pred_max,y_pred_min,y_pred_mean,y_pred_std,y_pred_place_0,y_pred_place_1,y_pred_place_2
0,0.080896,0.080896,0.005844,0.034071,0.040837,0.080896,0.015472,0.005844
1,0.015472,0.080896,0.005844,0.034071,0.040837,0.080896,0.015472,0.005844
2,0.005844,0.080896,0.005844,0.034071,0.040837,0.080896,0.015472,0.005844
3,0.143397,0.387849,0.143397,0.297128,0.133854,0.143397,0.387849,0.360138
4,0.387849,0.387849,0.143397,0.297128,0.133854,0.143397,0.387849,0.360138
5,0.360138,0.387849,0.143397,0.297128,0.133854,0.143397,0.387849,0.360138
6,0.000543,0.001063,0.000078,0.000561,0.000492,0.000543,0.001063,0.000078
7,0.001063,0.001063,0.000078,0.000561,0.000492,0.000543,0.001063,0.000078
8,0.000078,0.001063,0.000078,0.000561,0.000492,0.000543,0.001063,0.000078
9,0.000030,0.000068,0.000008,0.000035,0.000030,0.000030,0.000068,0.000008
